In [1]:
from collections import defaultdict

import torch
from torchvision import datasets as D, transforms as T
from tqdm import tqdm

import layer
from loss_function import MarginLoss

torch.manual_seed(0)
device = torch.device('mps' if torch.has_mps else 'cuda' if torch.has_cuda else 'cpu')

print(f'Using {device.type}')

/Users/zarizky/conda-envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using mps


In [2]:
train_dataset = D.MNIST('mnist', True, T.ToTensor(), download=True)
valid_dataset = D.MNIST('mnist', False, T.ToTensor())

batch_size = 32
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size, True, num_workers=8)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size, True, num_workers=8)

lr = 5e-4
model = torch.nn.Sequential(
    torch.nn.Conv2d(1, 256, 9),
    torch.nn.ReLU(),
    layer.PrimaryCapsule(256, 32, 8, 9, 2),
    layer.Squash('dr'),
    layer.CapsuleTransform(32 * 6 * 6, 8, 10, 16),
    layer.DynamicRouter(10, 16)
).to('mps')

print(f'Model size: {sum(p.numel() for p in model.parameters()):,}')

criterion = MarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr)

Model size: 6,804,384


In [3]:
print(f'CapsNet parameter size: {sum(p.numel() for p in model.parameters()):,}')
print()

history = []
epochs = 150
for i in range(1, epochs + 1):
    record = defaultdict(float)

    print(f'Epoch {i}/{epochs}')
    with tqdm(total=len(trainloader) + len(validloader)) as pbar:
        model.train()
        for inputs, targets in trainloader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs).norm(p=2, dim=-1)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            record['loss'] += loss.item()
            record['accuracy'] += (outputs.argmax(-1) == targets).float().mean().item() * 100
            pbar.update()

        record['loss'] /= len(trainloader)
        record['accuracy'] /= len(trainloader)
        pbar.set_postfix(record)

        model.eval()
        for inputs, targets in validloader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            with torch.inference_mode():
                outputs = model(inputs).norm(p=2, dim=-1)
                loss = criterion(outputs, targets)

            record['val_loss'] += loss.item()
            record['val_accuracy'] += (outputs.argmax(-1) == targets).float().mean().item() * 100
            pbar.update()wmm

        record['val_loss'] /= len(validloader)
        record['val_accuracy'] /= len(validloader)
        pbar.set_postfix(record)
        history.append(record)

CapsNet parameter size: 6,804,384

Epoch 1/150


100%|██████████| 2188/2188 [03:23<00:00, 10.77it/s, loss=0.221, accuracy=75.6, val_loss=0.245, val_accuracy=74.2]


Epoch 2/150


100%|██████████| 2188/2188 [03:26<00:00, 10.61it/s, loss=0.167, accuracy=82.8, val_loss=0.264, val_accuracy=77.7]


Epoch 3/150


...

Epoch 21/150


100%|██████████| 2188/2188 [03:36<00:00, 10.12it/s, loss=0.103, accuracy=90, val_loss=0.817, val_accuracy=54]


Epoch 22/150


 79%|███████▊  | 1719/2188 [03:05<00:50,  9.26it/s]


KeyboardInterrupt: 